In [1]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
import schedulers

import tensorflow as tf
import tensorflowjs as tfjs

assert tf.config.list_physical_devices('GPU')

In [2]:
BATCH_SIZE = 64

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)

def build_model(UNITS=500):
    inp = keras.Input(shape=(64,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, UNITS, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True), merge_mode='sum')(embed)
    layer = layers.add([layer, layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True), merge_mode='sum')(layer)])
    layer = layers.BatchNormalization()(layer)
    layer = layers.add([embed, layers.Dense(UNITS, activation='relu')(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    model = keras.Model(inputs=inp, outputs=outputs)

    return model

model = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 64, 500)      22000       input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 64, 500)      4004000     embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 64, 500)      4004000     bidirectional[0][0]              
______________________________________________________________________________________________

In [3]:

# masked version of accuracy and sce
def accuracy(real, pred):
    acc = tf.keras.metrics.sparse_categorical_accuracy(real, pred)

    mask = tf.cast(tf.math.logical_not(tf.math.equal(real, 0)), dtype=acc.dtype)
    acc *= mask

    return tf.reduce_sum(acc) / tf.reduce_sum(mask)

def sparse_categorical_crossentropy(y_true, y_pred, sample_weight=None):
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

    mask = tf.cast(tf.math.logical_not(tf.math.equal(y_true, 0)), dtype=loss.dtype)
    loss *= mask

    return tf.reduce_sum(loss) / tf.reduce_sum(mask) 

def fit(train_validation, scheduler=None, verbose=1, lr=1e-4, epochs=1):
    train, valid = train_validation
    
    model.compile(loss=sparse_categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics={'N': accuracy, 'D': accuracy, 'S': accuracy})
    callbacks = []
    if isinstance(scheduler, schedulers.CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)    
    
    x  = train.normalized
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin }
    
    if valid is None:
        return model.fit(x, y, batch_size=BATCH_SIZE, epochs=epochs, verbose=verbose, callbacks=callbacks)
    
    vx = valid.normalized
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin }
    
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=epochs, verbose=verbose, callbacks=callbacks)


MAXLEN = 64
def load_data(filenames, maxlen=MAXLEN, validation=0.1):
    if validation == 0:
        corpus = dataset.read_corpus(filenames, maxlen)
        train = dataset.Data.concatenate(corpus)
        train.shuffle()
        return train, None
    
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid


def load_test_data(maxlen=MAXLEN):
    return dataset.load_file('test/ModernTestCorpus-HebrewWiki1-simple.txt', maxlen) 


In [4]:
data_mix = load_data([
    'hebrew_diacritized_private/poetry',
    'hebrew_diacritized_private/rabanit',
    'hebrew_diacritized_private/pre_modern'], validation=0)

In [5]:
data_modern = load_data(['hebrew_diacritized/modern'], validation=0)

In [6]:
model.load_weights('./checkpoints/uninit')
history = fit(data_mix, scheduler=schedulers.CircularLearningRate(30e-4, 80e-4, 1e-4), epochs=1)
model.save_weights('./checkpoints/mix')

2426/2426 [==============================] - 244s 101ms/step - loss: 0.2490 - N_loss: 0.1908 - D_loss: 0.0517 - S_loss: 0.0065 - N_accuracy: 0.9329 - D_accuracy: 0.9792 - S_accuracy: 0.9980s - loss: 0.2538 - N_loss: 0.1946 -


In [7]:
model.load_weights('./checkpoints/mix')
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(50e-4, 50e-4, 1e-5))
model.save_weights('./checkpoints/modern')

292/292 [==============================] - 29s 100ms/step - loss: 0.1421 - N_loss: 0.1059 - D_loss: 0.0340 - S_loss: 0.0022 - N_accuracy: 0.9644 - D_accuracy: 0.9870 - S_accuracy: 0.999521s - loss: 0.1969 - N_loss: 0.1474 - D - ETA: 9s - loss: 0.1595 - N_loss: 0.1192 - 


In [8]:
model.load_weights('./checkpoints/modern')
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(5e-4, 15e-4, 1e-5), epochs=1)
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(5e-5, 1e-4, 1e-5), epochs=1)
model.save_weights('./checkpoints/modern_over')

292/292 [==============================] - 29s 100ms/step - loss: 0.0281 - N_loss: 0.0191 - D_loss: 0.0086 - S_loss: 4.3159e-04 - N_accuracy: 0.9938 - D_accuracy: 0.9968 - S_accuracy: 0.9999


In [24]:
model.load_weights('./checkpoints/modern_over')

model.compile()
model.save('modern.h5')
tfjs.converters.save_keras_model(model, '.')

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=3)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['N', 'D', 'S'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [9]:
model.load_weights('./checkpoints/modern_over')
test = load_test_data()
x = test.normalized
y = {'N': test.niqqud, 'D': test.dagesh, 'S': test.sin }

model.compile(loss=sparse_categorical_crossentropy,
              metrics={'N': accuracy, 'D': accuracy, 'S': accuracy})

model.evaluate(x=x, y=y, batch_size=BATCH_SIZE)

10/10 [==============================] - 0s 39ms/step - loss: 0.2131 - N_loss: 0.1708 - D_loss: 0.0389 - S_loss: 0.0034 - N_accuracy: 0.9557 - D_accuracy: 0.9878 - S_accuracy: 0.9991


[0.21313048899173737,
 0.1707678735256195,
 0.03894208371639252,
 0.0034205657429993153,
 0.9556616544723511,
 0.9878182411193848,
 0.999067485332489]

In [10]:
model.load_weights('./checkpoints/modern_over')

def real_evaluation(data, s=slice(0, None), print_comparison=True):
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud[s], data.dagesh[s], data.sin[s]]
    actual = dataset.merge(data.text[s], ts=batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(data.text[s], ts=batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    total_letters = []
    total_words = []
    for i, (b, a, e) in enumerate(zip(batch, actual, expected)):
        res = ( (expected_niqqud[i][b > 16] == actual_niqqud[i][b > 16])
              & (expected_dagesh[i][b > 16] == actual_dagesh[i][b > 16])
              & (expected_sin[i][b > 16] == actual_sin[i][b > 16]) )
        total_letters.extend(res)
        
        for aw, ew in zip(a.split(), e.split()):
            if len([x for x in 'אבגדהוזחטיכלמנסעפצקרשתךםןףץ' if x in aw]) > 1:
                total_words.append(aw == ew)
                if print_comparison and aw != ew:
                    print(aw, ew)
        
        if print_comparison:
            print('מצוי: ', a)
            print('רצוי: ', e)
            print(f'{np.mean(res):.2%} ({len(res)-np.sum(res)} out of {len(res)})')
            print()
    print(f'letters: {np.mean(total_letters):.2%}, words: {np.mean(total_words):.2%}')

real_evaluation(test, print_comparison=False)

letters: 92.83%, words: 78.85%
